In [41]:
# 추가되는 패키지는 여기에서 import 해주세요
import warnings
warnings.filterwarnings('ignore')

import missingno
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import geopy.distance
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)

import numpy as np
import datetime
import statsmodels.api as sm

In [42]:
# 데이터 load
train = pd.read_csv('train.csv', date_parser='date')
# 분석을 위해 컬럼명 수정
train.rename(columns={'6~7_ride': 'ride6', '7~8_ride': 'ride7', '8~9_ride': 'ride8',\
                      '9~10_ride': 'ride9', '10~11_ride': 'ride10', '11~12_ride': 'ride11', 
                      '6~7_takeoff': 'off6', '7~8_takeoff': 'off7', '8~9_takeoff': 'off8', \
                      '9~10_takeoff': 'off9', '10~11_takeoff': 'off10', '11~12_takeoff': 'off11',\
                      '18~20_ride': 'ride18'},inplace=True)


# 시작 날짜와, 끝 날짜 확인
print(train['date'].min())
print(train['date'].max())
# date 타입 변경
train['date'] = train['date'].astype('datetime64')
# 9월 1일 ~ 9월 30일 데이터이니까, day만 따로 가져옴
train['date_day'] = train['date'].dt.day
# 월(0) ~ 일(6) 데이터로 변환
train['date_dayofweek'] = train['date'].dt.dayofweek
train['date_dayofname']= train['date'].dt.day_name()
train[['date','date_dayofweek','date_day','date_dayofname']]

#평일과 휴일을 나누기, 금토일 =1, 나머지는 0
train.loc[train['date_dayofweek'] == 0, 'holiday' ] = 0
train.loc[train['date_dayofweek'] == 1, 'holiday' ] = 0
train.loc[train['date_dayofweek'] == 2, 'holiday' ] = 0
train.loc[train['date_dayofweek'] == 3, 'holiday' ] = 0
train.loc[train['date_dayofweek'] == 4, 'holiday' ] = 1
train.loc[train['date_dayofweek'] == 5, 'holiday' ] = 1
train.loc[train['date_dayofweek'] == 6, 'holiday' ] = 1
# 추석 추가도 공휴일로 처리
train.loc[train['date'] == '2019-09-14', 'holiday'] = 1
train.loc[train['date'] == '2019-09-13', 'holiday'] = 1
train.loc[train['date'] == '2019-09-12', 'holiday'] = 1


# 시내와 시외를 int형으로 원핫인코딩
train.loc[train['in_out'] == '시외','in_out'] = 1
train.loc[train['in_out'] == '시내','in_out'] = 0

train.tail()

2019-09-01
2019-09-30


,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday
415418,415418,2019-09-30,32820000,0,1129,한림환승정류장(한림리),33.41437,126.26336,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0
415419,415419,2019-09-30,32820000,0,1564,제주시외버스터미널,33.49946,126.51479,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0
415420,415420,2019-09-30,32820000,0,2322,해병부대,33.23100,126.26273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0
415421,415421,2019-09-30,32820000,0,3291,애월환승정류장(애월리),33.46483,126.31870,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0
415422,415422,2019-09-30,32820000,0,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0


In [43]:
total = train.groupby(['station_name', 'station_code']).count()
total

id  date  bus_route_id  in_out  latitude  \
station_name station_code                                              
(구)구판장       4211          33   33    33            33      33         
             4212          10   10    10            10      10         
(구)중문동주민센터   2058          263  263   263           263     263        
             2059          299  299   299           299     299        
(구)중앙파출소     1952          280  280   280           280     280        
             2080          177  177   177           177     177        
             2696          76   76    76            76      76         
(구)화산초등학교    2146          6    6     6             6       6          
             4210          18   18    18            18      18         
1100고지휴게소    1245          18   18    18            18      18         
             2638          25   25    25            25      25         
1100도로입구     1653          74   74    74            74      74         
4.3평화공원      1526          44   44    44            44      44         
             1527          10   10    10            10      10         
4.3평화공원입구    1529          14   14    14            14      14         
6호광장         694           162  162   162           162     162        
             695           66   66    66            66      66         
911의원        1243          46   46    46            46      46         
             1244          84   84    84            84      84         
GM빌라         538           61   61    61            61      61         
             539           68   68    68            68      68         
KIS빌리지       4086          5    5     5             5       5          
             4087          18   18    18            18      18         
LH12단지       2767          89   89    89            89      89         
             2768          124  124   124           124     124        
LH아파트        2083          464  464   464           464     464        
L마트          1937          413  413   413           413     413        
NHF삼화1단지     3510          379  379   379           379     379        
NLCS제주       2741          74   74    74            74      74         
             2742          72   72    72            72      72         
...                        ..   ..    ..            ..      ..         
회수마을회관       3073          137  137   137           137     137        
회수사거리        3590          22   22    22            22      22         
             3591          16   16    16            16      16         
회수삼거리        1649          69   69    69            69      69         
             1650          39   39    39            39      39         
회천입구         263           151  151   151           151     151        
효돈농협하나로마트    3389          93   93    93            93      93         
             3390          35   35    35            35      35         
효돈중학교        1817          248  248   248           248     248        
             1818          317  317   317           317     317        
효돈초등학교       1819          357  357   357           357     357        
             2072          259  259   259           259     259        
효돈축구공원       3387          22   22    22            22      22         
             3388          4    4     4             4       4          
효례교          2075          85   85    85            85      85         
효성빌라         1799          61   61    61            61      61         
             1800          53   53    53            53      53         
휴애리자연생활공원    3614          11   11    11            11      11         
             3615          20   20    20            20      20         
흙담솔사가        3478          23   23    23            23      23         
             3479          39   39    39            39      39         
흙통           1922          34   34    34            34      34         
             1923          46   46    46            46      46      

In [44]:
station = {}
for idx in total.index:
    if idx[0] not in station.keys():
        station[idx[0]] = 1
    else:
        station[idx[0]] += 1

df = pd.DataFrame()
df['station_name'] = station.keys()
df['counts'] = station.values()
df = df.sort_values(['counts','station_name'], ascending = [False, True])
df.head()

,station_name,counts
1246,신안동,8
1178,금악리,7
713,비석거리,6
1383,성읍1리,6
1654,산방산,5


In [45]:
total.tail()

id  date  bus_route_id  in_out  latitude  \
station_name station_code                                              
흥덕사          4199          11   11    11            11      11         
             4200          10   10    10            10      10         
희진주유소        2584          74   74    74            74      74         
             2585          102  102   102           102     102        
히귀반석         503           73   73    73            73      73         

                           longitude  ride6  ride7  ride8  ride9  ride10  \
station_name station_code                                                  
흥덕사          4199          11         11     11     11     11     11       
             4200          10         10     10     10     10     10       
희진주유소        2584          74         74     74     74     74     74       
             2585          102        102    102    102    102    102      
히귀반석         503           73         73     73     73     73     73       

                           ride11  off6  off7  off8  off9  off10  off11  \
station_name station_code                                                 
흥덕사          4199          11      11    11    11    11    11     11      
             4200          10      10    10    10    10    10     10      
희진주유소        2584          74      74    74    74    74    74     74      
             2585          102     102   102   102   102   102    102     
히귀반석         503           73      73    73    73    73    73     73      

                           ride18  date_day  date_dayofweek  date_dayofname  \
station_name station_code                                                     
흥덕사          4199          11      11        11              11               
             4200          10      10        10              10               
희진주유소        2584          74      74        74              74               
             2585          102     102       102             102              
히귀반석         503           73      73        73              73               

                           holiday  
station_name station_code           
흥덕사          4199          11       
             4200          10       
희진주유소        2584          74       
             2585          102      
히귀반석         503           73

In [46]:
coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
coords_seoquipo = (33.259429, 126.558217) #서귀포시의 위도 경도
train['dis_jejusi'] = [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_jejusi).km for i in range(len(train))]
train['dis_seoquipo'] = [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(train))]

In [47]:
def get_station_across():
    
#     d2 = train
    d2 = train.loc[0:500] ## 데이터 확인용 
    blank = pd.DataFrame()
    
    for i in d2.index:
        print(str(i)+"/"+ str(len(d2)))
        test = d2.loc[i]
        test_date = d2.loc[i, 'date']
        test_st_name = d2.loc[i, 'station_name']
        test_dis = d2.loc[i, 'dis_jejusi']       
        

        right = d2.query('date == @test_date & station_name == @test_st_name').reset_index()
        right['code'] = abs(right['station_code'] - d2.loc[i, 'station_code'])
        right['standard'] = abs(right['dis_jejusi'] - d2.loc[i, 'dis_jejusi'])

        bag = pd.DataFrame(np.zeros(len(right.columns))).T #26
        bag.columns = right.columns
        bag.station_name = right.station_name
        bag.station_code = right.station_code
        bag.date = right.date
        bag.in_out = right.in_out
        bag.bus_route_id = right.bus_route_id

        
        try:
            A = pd.DataFrame(test).T.merge(pd.DataFrame(right.sort_values(['code','standard']).reset_index(drop = True).loc[1]).T, on = 'station_name')
        except:
            A = pd.DataFrame(test).T.merge(bag, on = 'station_name')
        
        blank = pd.concat([blank, A])
    blank.drop(['code','standard','index','id_y'], axis = 1, inplace = True)
    blank.reset_index(inplace = True)
    return blank

In [48]:
new_train = get_station_across()
# new_train.to_csv('newtrain.csv')
new_train[['date_x','station_code_x','station_name','date_y','station_code_y']]

0/501
1/501
2/501
3/501
4/501
5/501
6/501
7/501
8/501
9/501
10/501
11/501
12/501
13/501
14/501
15/501
16/501
17/501
18/501
19/501
20/501
21/501
22/501
23/501
24/501
25/501
26/501
27/501
28/501
29/501
30/501
31/501
32/501
33/501
34/501
35/501
36/501
37/501
38/501
39/501
40/501
41/501
42/501
43/501
44/501
45/501
46/501
47/501
48/501
49/501
50/501
51/501
52/501
53/501
54/501
55/501
56/501
57/501
58/501
59/501
60/501
61/501
62/501
63/501
64/501
65/501
66/501
67/501
68/501
69/501
70/501
71/501
72/501
73/501
74/501
75/501
76/501
77/501
78/501
79/501
80/501
81/501
82/501
83/501
84/501
85/501
86/501
87/501
88/501
89/501
90/501
91/501
92/501
93/501
94/501
95/501
96/501
97/501
98/501
99/501
100/501
101/501
102/501
103/501
104/501
105/501
106/501
107/501
108/501
109/501
110/501
111/501
112/501
113/501
114/501
115/501
116/501
117/501
118/501
119/501
120/501
121/501
122/501
123/501
124/501
125/501
126/501
127/501
128/501
129/501
130/501
131/501
132/501
133/501
134/501
135/501
136/501
137/501
138/50

,date_x,station_code_x,station_name,date_y,station_code_y
0,2019-09-01 00:00:00,344,제주썬호텔,2019-09-01 00:00:00,343
1,2019-09-01 00:00:00,357,한라병원,2019-09-01 00:00:00,357
2,2019-09-01 00:00:00,432,정존마을,2019-09-01 00:00:00,432
3,2019-09-01 00:00:00,1579,제주국제공항(600번),2019-09-01 00:00:00,1579
4,2019-09-01 00:00:00,1646,중문관광단지입구,2019-09-01 00:00:00,1645
5,2019-09-01 00:00:00,1648,예래입구,2019-09-01 00:00:00,1647
6,2019-09-01 00:00:00,1660,제주국제컨벤션센터중문대포해안주상절리대,2019-09-01 00:00:00,1660
7,2019-09-01 00:00:00,1666,창천리,2019-09-01 00:00:00,1666
8,2019-09-01 00:00:00,1726,강정농협,2019-09-01 00:00:00,1726
9,2019-09-01 00:00:00,2008,서귀포항,2019-09-01 00:00:00,2105


In [ ]:
new_train.to_csv('newtrain.csv')

In [50]:
train.query('date == "2019-09-01" & station_name == "정존마을"')

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday,dis_jejusi,dis_seoquipo
2,2,2019-09-01,4270000,1,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1,6,Sunday,1.0,5.036124,25.893305
27,27,2019-09-01,4280000,0,431,정존마을,33.48132,126.47330,0.0,2.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1,6,Sunday,1.0,5.077571,25.847800
61,61,2019-09-01,8170000,1,432,정존마을,33.48181,126.47352,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,5.036124,25.893305
70,70,2019-09-01,8180000,0,431,정존마을,33.48132,126.47330,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,6,Sunday,1.0,5.077571,25.847800
337,337,2019-09-01,21410000,1,432,정존마을,33.48181,126.47352,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,5.036124,25.893305
349,349,2019-09-01,21420000,0,431,정존마을,33.48132,126.47330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,1,6,Sunday,1.0,5.077571,25.847800
396,396,2019-09-01,21470000,0,431,정존마을,33.48132,126.47330,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,5.077571,25.847800
411,411,2019-09-01,21480000,0,431,정존마을,33.48132,126.47330,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,1.0,3.0,1.0,1,6,Sunday,1.0,5.077571,25.847800
444,444,2019-09-01,21520000,1,432,정존마을,33.48181,126.47352,0.0,2.0,2.0,4.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,1,6,Sunday,1.0,5.036124,25.893305
553,553,2019-09-01,21880000,1,432,정존마을,33.48181,126.47352,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,5.036124,25.893305


In [2]:
train.head()

NameError: name 'train' is not defined

In [6]:
train.query('date == "2019-09-01" & station_name == "제주썬호텔"')


,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday
0,0,2019-09-01,4270000,1,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0
25,25,2019-09-01,4280000,1,343,제주썬호텔,33.48972,126.49370,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1,6,Sunday,1.0


In [9]:
coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
coords_seoquipo = (33.259429, 126.558217) #서귀포시의 위도 경도
train['dis_jejusi'] = [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_jejusi).km for i in range(len(train))]
train['dis_seoquipo'] = [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(train))]

In [51]:
d2 = train.loc[0:500] ## 데이터 확인용 
blank = pd.DataFrame()

test = d2.loc[27]
test_date = d2.loc[27, 'date']
test_st_name = d2.loc[27, 'station_name']
test_dis = d2.loc[27, 'dis_jejusi']       


right = d2.query('date == @test_date & station_name == @test_st_name').reset_index()
right['code'] = abs(right['station_code'] - d2.loc[27, 'station_code'])
right['standard'] = abs(right['dis_jejusi'] - d2.loc[27, 'dis_jejusi'])
A = pd.DataFrame(test).T.merge(pd.DataFrame(right.sort_values(['code','standard']).reset_index(drop = True).loc[1]).T, on = 'station_name')

In [60]:
tmp = pd.DataFrame(right.sort_values(['code','standard']).reset_index(drop = True))
tmp

,index,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday,dis_jejusi,dis_seoquipo,code,standard
0,27,27,2019-09-01,4280000,0,431,정존마을,33.48132,126.47330,0.0,2.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1,6,Sunday,1.0,5.077571,25.847800,0,0.000000
1,70,70,2019-09-01,8180000,0,431,정존마을,33.48132,126.47330,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,6,Sunday,1.0,5.077571,25.847800,0,0.000000
2,349,349,2019-09-01,21420000,0,431,정존마을,33.48132,126.47330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,1,6,Sunday,1.0,5.077571,25.847800,0,0.000000
3,396,396,2019-09-01,21470000,0,431,정존마을,33.48132,126.47330,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,5.077571,25.847800,0,0.000000
4,411,411,2019-09-01,21480000,0,431,정존마을,33.48132,126.47330,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,1.0,3.0,1.0,1,6,Sunday,1.0,5.077571,25.847800,0,0.000000
5,2,2,2019-09-01,4270000,1,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1,6,Sunday,1.0,5.036124,25.893305,1,0.041447
6,61,61,2019-09-01,8170000,1,432,정존마을,33.48181,126.47352,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,5.036124,25.893305,1,0.041447
7,337,337,2019-09-01,21410000,1,432,정존마을,33.48181,126.47352,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,5.036124,25.893305,1,0.041447
8,444,444,2019-09-01,21520000,1,432,정존마을,33.48181,126.47352,0.0,2.0,2.0,4.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,1,6,Sunday,1.0,5.036124,25.893305,1,0.041447


In [38]:
pd.DataFrame(right.sort_values(['code','standard']).reset_index(drop = True).loc[1]).T

,index,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday,dis_jejusi,dis_seoquipo,code,standard
1,0,0,2019-09-01 00:00:00,4270000,1,344,제주썬호텔,33.4899,126.494,0,1,2,5,2,6,0,0,0,0,0,0,0,1,6,Sunday,1,2.95492,26.2567,1,0.0107426


In [58]:
right

,index,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday,dis_jejusi,dis_seoquipo,code,standard
0,2,2,2019-09-01,4270000,1,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1,6,Sunday,1.0,5.036124,25.893305,1,0.041447
1,27,27,2019-09-01,4280000,0,431,정존마을,33.48132,126.47330,0.0,2.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1,6,Sunday,1.0,5.077571,25.847800,0,0.000000
2,61,61,2019-09-01,8170000,1,432,정존마을,33.48181,126.47352,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,5.036124,25.893305,1,0.041447
3,70,70,2019-09-01,8180000,0,431,정존마을,33.48132,126.47330,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,6,Sunday,1.0,5.077571,25.847800,0,0.000000
4,337,337,2019-09-01,21410000,1,432,정존마을,33.48181,126.47352,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,5.036124,25.893305,1,0.041447
5,349,349,2019-09-01,21420000,0,431,정존마을,33.48132,126.47330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,1,6,Sunday,1.0,5.077571,25.847800,0,0.000000
6,396,396,2019-09-01,21470000,0,431,정존마을,33.48132,126.47330,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,5.077571,25.847800,0,0.000000
7,411,411,2019-09-01,21480000,0,431,정존마을,33.48132,126.47330,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,1.0,3.0,1.0,1,6,Sunday,1.0,5.077571,25.847800,0,0.000000
8,444,444,2019-09-01,21520000,1,432,정존마을,33.48181,126.47352,0.0,2.0,2.0,4.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,1,6,Sunday,1.0,5.036124,25.893305,1,0.041447


In [ ]:
total.reset_index()

In [478]:
total.query('station_name == "서귀포시외버스터미널"')

,,id,date,bus_route_id,in_out,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
station_name,station_code,,,,,,,,,,,,,,,,,,,
서귀포시외버스터미널,6115100,440,440,440,440,440,440,440,440,440,440,440,440,440,440,440,440,440,440,440


In [475]:
d2 = train.query('date == "2019-09-01"') ## 데이터 확인용 
blank = pd.DataFrame()

for i in d2.index:
    print(str(i)+"/"+ str(len(d2)))
    test = d2.loc[i]
    test_date = d2.loc[i, 'date']
    test_st_name = d2.loc[i, 'station_name']
    test_dis = d2.loc[i, 'dis_jejusi']       
    
    right = d2.query('date == @test_date & station_name == @test_st_name').reset_index()
    right['code'] = abs(right['station_code'] - d2.loc[i, 'station_code'])
    right['standard'] = abs(right['dis_jejusi'] - d2.loc[i, 'dis_jejusi'])
    bag = pd.DataFrame(np.zeros(len(right.columns))).T #26
    bag.columns = right.columns
    bag.station_name = right.station_name
    bag.station_code = right.station_code
    bag.date = right.date
    bag.in_out = right.in_out
    bag.bus_route_id = right.bus_route_id

0/11538
1/11538
2/11538
3/11538
4/11538
5/11538
6/11538
7/11538
8/11538
9/11538
10/11538
11/11538
12/11538
13/11538
14/11538
15/11538
16/11538
17/11538
18/11538
19/11538
20/11538
21/11538
22/11538
23/11538
24/11538
25/11538
26/11538
27/11538
28/11538
29/11538
30/11538
31/11538
32/11538
33/11538
34/11538
35/11538
36/11538
37/11538
38/11538
39/11538
40/11538
41/11538
42/11538
43/11538
44/11538
45/11538
46/11538
47/11538
48/11538
49/11538
50/11538
51/11538
52/11538
53/11538
54/11538
55/11538
56/11538
57/11538
58/11538
59/11538
60/11538
61/11538
62/11538
63/11538
64/11538
65/11538
66/11538
67/11538
68/11538
69/11538
70/11538
71/11538
72/11538
73/11538
74/11538
75/11538
76/11538
77/11538
78/11538
79/11538
80/11538
81/11538
82/11538
83/11538
84/11538
85/11538
86/11538
87/11538
88/11538
89/11538
90/11538
91/11538
92/11538
93/11538
94/11538
95/11538
96/11538
97/11538
98/11538
99/11538
100/11538
101/11538
102/11538
103/11538
104/11538
105/11538
106/11538
107/11538
108/11538
109/11538
110/11538


845/11538
846/11538
847/11538
848/11538
849/11538
850/11538
851/11538
852/11538
853/11538
854/11538
855/11538
856/11538
857/11538
858/11538
859/11538
860/11538
861/11538
862/11538
863/11538
864/11538
865/11538
866/11538
867/11538
868/11538
869/11538
870/11538
871/11538
872/11538
873/11538
874/11538
875/11538
876/11538
877/11538
878/11538
879/11538
880/11538
881/11538
882/11538
883/11538
884/11538
885/11538
886/11538
887/11538
888/11538
889/11538
890/11538
891/11538
892/11538
893/11538
894/11538
895/11538
896/11538
897/11538
898/11538
899/11538
900/11538
901/11538
902/11538
903/11538
904/11538
905/11538
906/11538
907/11538
908/11538
909/11538
910/11538
911/11538
912/11538
913/11538
914/11538
915/11538
916/11538
917/11538
918/11538
919/11538
920/11538
921/11538
922/11538
923/11538
924/11538
925/11538
926/11538
927/11538
928/11538
929/11538
930/11538
931/11538
932/11538
933/11538
934/11538
935/11538
936/11538
937/11538
938/11538
939/11538
940/11538
941/11538
942/11538
943/11538
944/11538


1616/11538
1617/11538
1618/11538
1619/11538
1620/11538
1621/11538
1622/11538
1623/11538
1624/11538
1625/11538
1626/11538
1627/11538
1628/11538
1629/11538
1630/11538
1631/11538
1632/11538
1633/11538
1634/11538
1635/11538
1636/11538
1637/11538
1638/11538
1639/11538
1640/11538
1641/11538
1642/11538
1643/11538
1644/11538
1645/11538
1646/11538
1647/11538
1648/11538
1649/11538
1650/11538
1651/11538
1652/11538
1653/11538
1654/11538
1655/11538
1656/11538
1657/11538
1658/11538
1659/11538
1660/11538
1661/11538
1662/11538
1663/11538
1664/11538
1665/11538
1666/11538
1667/11538
1668/11538
1669/11538
1670/11538
1671/11538
1672/11538
1673/11538
1674/11538
1675/11538
1676/11538
1677/11538
1678/11538
1679/11538
1680/11538
1681/11538
1682/11538
1683/11538
1684/11538
1685/11538
1686/11538
1687/11538
1688/11538
1689/11538
1690/11538
1691/11538
1692/11538
1693/11538
1694/11538
1695/11538
1696/11538
1697/11538
1698/11538
1699/11538
1700/11538
1701/11538
1702/11538
1703/11538
1704/11538
1705/11538
1706/11538

2371/11538
2372/11538
2373/11538
2374/11538
2375/11538
2376/11538
2377/11538
2378/11538
2379/11538
2380/11538
2381/11538
2382/11538
2383/11538
2384/11538
2385/11538
2386/11538
2387/11538
2388/11538
2389/11538
2390/11538
2391/11538
2392/11538
2393/11538
2394/11538
2395/11538
2396/11538
2397/11538
2398/11538
2399/11538
2400/11538
2401/11538
2402/11538
2403/11538
2404/11538
2405/11538
2406/11538
2407/11538
2408/11538
2409/11538
2410/11538
2411/11538
2412/11538
2413/11538
2414/11538
2415/11538
2416/11538
2417/11538
2418/11538
2419/11538
2420/11538
2421/11538
2422/11538
2423/11538
2424/11538
2425/11538
2426/11538
2427/11538
2428/11538
2429/11538
2430/11538
2431/11538
2432/11538
2433/11538
2434/11538
2435/11538
2436/11538
2437/11538
2438/11538
2439/11538
2440/11538
2441/11538
2442/11538
2443/11538
2444/11538
2445/11538
2446/11538
2447/11538
2448/11538
2449/11538
2450/11538
2451/11538
2452/11538
2453/11538
2454/11538
2455/11538
2456/11538
2457/11538
2458/11538
2459/11538
2460/11538
2461/11538

3116/11538
3117/11538
3118/11538
3119/11538
3120/11538
3121/11538
3122/11538
3123/11538
3124/11538
3125/11538
3126/11538
3127/11538
3128/11538
3129/11538
3130/11538
3131/11538
3132/11538
3133/11538
3134/11538
3135/11538
3136/11538
3137/11538
3138/11538
3139/11538
3140/11538
3141/11538
3142/11538
3143/11538
3144/11538
3145/11538
3146/11538
3147/11538
3148/11538
3149/11538
3150/11538
3151/11538
3152/11538
3153/11538
3154/11538
3155/11538
3156/11538
3157/11538
3158/11538
3159/11538
3160/11538
3161/11538
3162/11538
3163/11538
3164/11538
3165/11538
3166/11538
3167/11538
3168/11538
3169/11538
3170/11538
3171/11538
3172/11538
3173/11538
3174/11538
3175/11538
3176/11538
3177/11538
3178/11538
3179/11538
3180/11538
3181/11538
3182/11538
3183/11538
3184/11538
3185/11538
3186/11538
3187/11538
3188/11538
3189/11538
3190/11538
3191/11538
3192/11538
3193/11538
3194/11538
3195/11538
3196/11538
3197/11538
3198/11538
3199/11538
3200/11538
3201/11538
3202/11538
3203/11538
3204/11538
3205/11538
3206/11538

...

8385/11538
8386/11538
8387/11538
8388/11538
8389/11538
8390/11538
8391/11538
8392/11538
8393/11538
8394/11538
8395/11538
8396/11538
8397/11538
8398/11538
8399/11538
8400/11538
8401/11538
8402/11538
8403/11538
8404/11538
8405/11538
8406/11538
8407/11538
8408/11538
8409/11538
8410/11538
8411/11538
8412/11538
8413/11538
8414/11538
8415/11538
8416/11538
8417/11538
8418/11538
8419/11538
8420/11538
8421/11538
8422/11538
8423/11538
8424/11538
8425/11538
8426/11538
8427/11538
8428/11538
8429/11538
8430/11538
8431/11538
8432/11538
8433/11538
8434/11538
8435/11538
8436/11538
8437/11538
8438/11538
8439/11538
8440/11538
8441/11538
8442/11538
8443/11538
8444/11538
8445/11538
8446/11538
8447/11538
8448/11538
8449/11538
8450/11538
8451/11538
8452/11538
8453/11538
8454/11538
8455/11538
8456/11538
8457/11538
8458/11538
8459/11538
8460/11538
8461/11538
8462/11538
8463/11538
8464/11538
8465/11538
8466/11538
8467/11538
8468/11538
8469/11538
8470/11538
8471/11538
8472/11538
8473/11538
8474/11538
8475/11538

9150/11538
9151/11538
9152/11538
9153/11538
9154/11538
9155/11538
9156/11538
9157/11538
9158/11538
9159/11538
9160/11538
9161/11538
9162/11538
9163/11538
9164/11538
9165/11538
9166/11538
9167/11538
9168/11538
9169/11538
9170/11538
9171/11538
9172/11538
9173/11538
9174/11538
9175/11538
9176/11538
9177/11538
9178/11538
9179/11538
9180/11538
9181/11538
9182/11538
9183/11538
9184/11538
9185/11538
9186/11538
9187/11538
9188/11538
9189/11538
9190/11538
9191/11538
9192/11538
9193/11538
9194/11538
9195/11538
9196/11538
9197/11538
9198/11538
9199/11538
9200/11538
9201/11538
9202/11538
9203/11538
9204/11538
9205/11538
9206/11538
9207/11538
9208/11538
9209/11538
9210/11538
9211/11538
9212/11538
9213/11538
9214/11538
9215/11538
9216/11538
9217/11538
9218/11538
9219/11538
9220/11538
9221/11538
9222/11538
9223/11538
9224/11538
9225/11538
9226/11538
9227/11538
9228/11538
9229/11538
9230/11538
9231/11538
9232/11538
9233/11538
9234/11538
9235/11538
9236/11538
9237/11538
9238/11538
9239/11538
9240/11538

9900/11538
9901/11538
9902/11538
9903/11538
9904/11538
9905/11538
9906/11538
9907/11538
9908/11538
9909/11538
9910/11538
9911/11538
9912/11538
9913/11538
9914/11538
9915/11538
9916/11538
9917/11538
9918/11538
9919/11538
9920/11538
9921/11538
9922/11538
9923/11538
9924/11538
9925/11538
9926/11538
9927/11538
9928/11538
9929/11538
9930/11538
9931/11538
9932/11538
9933/11538
9934/11538
9935/11538
9936/11538
9937/11538
9938/11538
9939/11538
9940/11538
9941/11538
9942/11538
9943/11538
9944/11538
9945/11538
9946/11538
9947/11538
9948/11538
9949/11538
9950/11538
9951/11538
9952/11538
9953/11538
9954/11538
9955/11538
9956/11538
9957/11538
9958/11538
9959/11538
9960/11538
9961/11538
9962/11538
9963/11538
9964/11538
9965/11538
9966/11538
9967/11538
9968/11538
9969/11538
9970/11538
9971/11538
9972/11538
9973/11538
9974/11538
9975/11538
9976/11538
9977/11538
9978/11538
9979/11538
9980/11538
9981/11538
9982/11538
9983/11538
9984/11538
9985/11538
9986/11538
9987/11538
9988/11538
9989/11538
9990/11538

10593/11538
10594/11538
10595/11538
10596/11538
10597/11538
10598/11538
10599/11538
10600/11538
10601/11538
10602/11538
10603/11538
10604/11538
10605/11538
10606/11538
10607/11538
10608/11538
10609/11538
10610/11538
10611/11538
10612/11538
10613/11538
10614/11538
10615/11538
10616/11538
10617/11538
10618/11538
10619/11538
10620/11538
10621/11538
10622/11538
10623/11538
10624/11538
10625/11538
10626/11538
10627/11538
10628/11538
10629/11538
10630/11538
10631/11538
10632/11538
10633/11538
10634/11538
10635/11538
10636/11538
10637/11538
10638/11538
10639/11538
10640/11538
10641/11538
10642/11538
10643/11538
10644/11538
10645/11538
10646/11538
10647/11538
10648/11538
10649/11538
10650/11538
10651/11538
10652/11538
10653/11538
10654/11538
10655/11538
10656/11538
10657/11538
10658/11538
10659/11538
10660/11538
10661/11538
10662/11538
10663/11538
10664/11538
10665/11538
10666/11538
10667/11538
10668/11538
10669/11538
10670/11538
10671/11538
10672/11538
10673/11538
10674/11538
10675/11538
1067

11278/11538
11279/11538
11280/11538
11281/11538
11282/11538
11283/11538
11284/11538
11285/11538
11286/11538
11287/11538
11288/11538
11289/11538
11290/11538
11291/11538
11292/11538
11293/11538
11294/11538
11295/11538
11296/11538
11297/11538
11298/11538
11299/11538
11300/11538
11301/11538
11302/11538
11303/11538
11304/11538
11305/11538
11306/11538
11307/11538
11308/11538
11309/11538
11310/11538
11311/11538
11312/11538
11313/11538
11314/11538
11315/11538
11316/11538
11317/11538
11318/11538
11319/11538
11320/11538
11321/11538
11322/11538
11323/11538
11324/11538
11325/11538
11326/11538
11327/11538
11328/11538
11329/11538
11330/11538
11331/11538
11332/11538
11333/11538
11334/11538
11335/11538
11336/11538
11337/11538
11338/11538
11339/11538
11340/11538
11341/11538
11342/11538
11343/11538
11344/11538
11345/11538
11346/11538
11347/11538
11348/11538
11349/11538
11350/11538
11351/11538
11352/11538
11353/11538
11354/11538
11355/11538
11356/11538
11357/11538
11358/11538
11359/11538
11360/11538
1136

In [477]:
d2.tail()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,dis_jejusi,dis_seoquipo
11533,11533,2019-09-01,32810000,시내,1129,한림환승정류장(한림리),33.41437,126.26336,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,25.950139,32.385460
11534,11534,2019-09-01,32820000,시내,96,제주민속오일장,33.49376,126.47670,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.350283,27.074266
11535,11535,2019-09-01,32820000,시내,1129,한림환승정류장(한림리),33.41437,126.26336,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,25.950139,32.385460
11536,11536,2019-09-01,32820000,시내,1564,제주시외버스터미널,33.49946,126.51479,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.754801,26.927069
11537,11537,2019-09-01,32820000,시내,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,27.987818,4.828453


In [476]:
right

,index,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,dis_jejusi,dis_seoquipo,code,standard
0,67,67,2019-09-01,8170000,시내,6115100,서귀포시외버스터미널,33.24873,126.50799,8.0,0.0,0.0,0.0,9.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,1.0,27.987818,4.828453,0,0.0
1,111,111,2019-09-01,21000000,시내,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,0.0,27.987818,4.828453,0,0.0
2,134,134,2019-09-01,21020000,시내,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,27.987818,4.828453,0,0.0
3,143,143,2019-09-01,21040000,시내,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,27.987818,4.828453,0,0.0
4,194,194,2019-09-01,21100000,시내,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,27.987818,4.828453,0,0.0
5,1233,1233,2019-09-01,22520000,시내,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,7.0,9.0,5.0,7.0,0.0,27.987818,4.828453,0,0.0
6,6126,6126,2019-09-01,28480000,시내,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,27.987818,4.828453,0,0.0
7,6403,6403,2019-09-01,28500000,시내,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,7.0,1.0,0.0,27.987818,4.828453,0,0.0
8,6620,6620,2019-09-01,28520000,시내,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,27.987818,4.828453,0,0.0
9,6729,6729,2019-09-01,28540000,시내,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,27.987818,4.828453,0,0.0


In [344]:
firsts.to_csv('even.csv')

In [342]:
firsts = even_station()

In [ ]:
d2 = train.copy()
d2.

In [389]:
test = d2.loc[3]
test_date = d2.loc[3, 'date']
test_st_name = d2.loc[3, 'station_name']
test_dis = d2.loc[3, 'dis_jejusi']

In [390]:
test_date, test_st_name, test_dis

('2019-09-01', '제주국제공항(600번)', 2.8641661537765564)

In [416]:
blank = pd.DataFrame()
bag = pd.DataFrame(np.zeros(26)).T
bag.columns = right.columns
bag.station_name = right.station_name

right = d2.query('date == @test_date & station_name == @test_st_name').reset_index()
right['code'] = abs(right['station_code'] - d2.loc[3, 'station_code'])
right['standard'] = abs(right['dis_jejusi'] - d2.loc[3, 'dis_jejusi'])
try:
    A = pd.DataFrame(test).T.merge(pd.DataFrame(right.sort_values(['code','standard']).loc[1]).T, on = 'station_name')
except:
    A = pd.DataFrame(test).T.merge(bag, on = 'station_name')
blank = pd.concat([blank, A])

In [403]:
test

id               3           
date             2019-09-01  
bus_route_id     4270000     
in_out           시내          
station_code     1579        
station_name     제주국제공항(600번)
latitude         33.5058     
longitude        126.493     
6~7_ride         0           
7~8_ride         17          
8~9_ride         6           
9~10_ride        26          
10~11_ride       14          
11~12_ride       16          
6~7_takeoff      0           
7~8_takeoff      0           
8~9_takeoff      0           
9~10_takeoff     0           
10~11_takeoff    0           
11~12_takeoff    0           
18~20_ride       53          
dis_jejusi       2.86417     
dis_seoquipo     27.9975     
Name: 3, dtype: object

In [417]:
blank

,id_x,date_x,bus_route_id_x,in_out_x,station_code_x,station_name,latitude_x,longitude_x,6~7_ride_x,7~8_ride_x,...,7~8_takeoff_y,8~9_takeoff_y,9~10_takeoff_y,10~11_takeoff_y,11~12_takeoff_y,18~20_ride_y,dis_jejusi_y,dis_seoquipo_y,code,standard
0,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.5058,126.493,0,17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [402]:
right

,index,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,...,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,dis_jejusi,dis_seoquipo,code,standard
0,3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,...,0.0,0.0,0.0,0.0,0.0,53.0,2.864166,27.997494,0,0.0


In [415]:
bag = pd.DataFrame(np.zeros(26)).T
bag.columns = right.columns
bag.station_name = right.station_name
c

,index,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,...,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,dis_jejusi,dis_seoquipo,code,standard
0,0.0,0.0,0.0,0.0,0.0,0.0,제주국제공항(600번),0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [392]:
right

,index,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,...,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,dis_jejusi,dis_seoquipo,code,standard
0,3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,...,0.0,0.0,0.0,0.0,0.0,53.0,2.864166,27.997494,0,0.0


In [395]:
right.station_name

0    제주국제공항(600번)
Name: station_name, dtype: object

In [ ]:
blank[['station_name','station_code_x','station_code_y']]

In [396]:
total.query('station_name == "제주국제공항(600번)"')

,,id,date,bus_route_id,in_out,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
station_name,station_code,,,,,,,,,,,,,,,,,,,
제주국제공항(600번),1579,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31


In [398]:
#하나면 어떡할래?
total

id  date  bus_route_id  in_out  latitude  \
station_name station_code                                              
(구)구판장       4211          33   33    33            33      33         
             4212          10   10    10            10      10         
(구)중문동주민센터   2058          263  263   263           263     263        
             2059          299  299   299           299     299        
(구)중앙파출소     1952          280  280   280           280     280        
             2080          177  177   177           177     177        
             2696          76   76    76            76      76         
(구)화산초등학교    2146          6    6     6             6       6          
             4210          18   18    18            18      18         
1100고지휴게소    1245          18   18    18            18      18         
             2638          25   25    25            25      25         
1100도로입구     1653          74   74    74            74      74         
4.3평화공원      1526          44   44    44            44      44         
             1527          10   10    10            10      10         
4.3평화공원입구    1529          14   14    14            14      14         
6호광장         694           162  162   162           162     162        
             695           66   66    66            66      66         
911의원        1243          46   46    46            46      46         
             1244          84   84    84            84      84         
GM빌라         538           61   61    61            61      61         
             539           68   68    68            68      68         
KIS빌리지       4086          5    5     5             5       5          
             4087          18   18    18            18      18         
LH12단지       2767          89   89    89            89      89         
             2768          124  124   124           124     124        
LH아파트        2083          464  464   464           464     464        
L마트          1937          413  413   413           413     413        
NHF삼화1단지     3510          379  379   379           379     379        
NLCS제주       2741          74   74    74            74      74         
             2742          72   72    72            72      72         
...                        ..   ..    ..            ..      ..         
회수마을회관       3073          137  137   137           137     137        
회수사거리        3590          22   22    22            22      22         
             3591          16   16    16            16      16         
회수삼거리        1649          69   69    69            69      69         
             1650          39   39    39            39      39         
회천입구         263           151  151   151           151     151        
효돈농협하나로마트    3389          93   93    93            93      93         
             3390          35   35    35            35      35         
효돈중학교        1817          248  248   248           248     248        
             1818          317  317   317           317     317        
효돈초등학교       1819          357  357   357           357     357        
             2072          259  259   259           259     259        
효돈축구공원       3387          22   22    22            22      22         
             3388          4    4     4             4       4          
효례교          2075          85   85    85            85      85         
효성빌라         1799          61   61    61            61      61         
             1800          53   53    53            53      53         
휴애리자연생활공원    3614          11   11    11            11      11         
             3615          20   20    20            20      20         
흙담솔사가        3478          23   23    23            23      23         
             3479          39   39    39            39      39         
흙통           1922          34   34    34            34      34         
             1923          46   46    46            46      46      

In [399]:
hard

,station_name,counts
1541,금악리,7
1093,산방산,5
1580,산양리,5
570,신시가지,5
242,조수1리,5
1152,(구)중앙파출소,3
820,고산동산,3
1898,교래리,3
1847,남조로검문소,3
742,납읍리,3
